# Week4: Getting Started with pandas

In [5]:
import pandas as pd

In [6]:
from pandas import Series, DataFrame

In [7]:
import numpy as np
np.random.seed(12345)
import matplotlib.pyplot as plt
plt.rc('figure', figsize=(10, 6))
PREVIOUS_MAX_ROWS = pd.options.display.max_rows
pd.options.display.max_rows = 20
np.set_printoptions(precision=4, suppress=True)

### Index Objects

Pandas's index objects are deisgned to contain the information of the axis labels og the table. Earlier we saw the default index based on a range function but any array of labels could be as well assigned to an index.

In [8]:
obj = pd.Series(range(3), index=['a', 'b', 'c'])
index = obj.index
index
index[1:]

Index(['b', 'c'], dtype='object')

index[1] = 'd'  # TypeError

notice that index objects are immutable. It is designed that way to make safer when it sahred among several data structures.

In [9]:
labels = pd.Index(np.arange(3))
labels
obj2 = pd.Series([1.5, -2.5, 0], index=labels)
obj2
obj2.index is labels

True

Pandas indexes can contain dupicate labels:

In [10]:
dup_labels = pd.Index(['foo', 'foo', 'bar', 'bar'])
dup_labels

Index(['foo', 'foo', 'bar', 'bar'], dtype='object')

## Essential Functionality

### Reindexing

__reindexing__ is an important method that is provided by Pandas. It allows to rearrange the ros of the DataFrame. In case a new label is provided the corresponding value will default to NaN.

In [11]:
obj = pd.Series([4.5, 7.2, -5.3, 3.6], index=['d', 'b', 'a', 'c'])
obj

d    4.5
b    7.2
a   -5.3
c    3.6
dtype: float64

In [12]:
obj2 = obj.reindex(['a', 'b', 'c', 'd', 'e'])
obj2

a   -5.3
b    7.2
c    3.6
d    4.5
e    NaN
dtype: float64

An interesting feature of __reindex__ method is to use the ffill optional argument that allows for some sort of an interpolation in the case of missing values.

In [13]:
obj3 = pd.Series(['blue', 'purple', 'yellow'], index=[0, 2, 4])
obj3


0      blue
2    purple
4    yellow
dtype: object

In [14]:
obj3.reindex(range(6), method='ffill')

0      blue
1      blue
2    purple
3    purple
4    yellow
5    yellow
dtype: object

The reindex function allows to rearrange the rows as well as the columns:

In [15]:
frame = pd.DataFrame(np.arange(9).reshape((3, 3)),
                     index=['a', 'c', 'd'],
                     columns=['Ohio', 'Texas', 'California'])
frame


,Ohio,Texas,California
a,0,1,2
c,3,4,5
d,6,7,8


In [16]:
frame2 = frame.reindex(['a', 'b', 'c', 'd'])
frame2

,Ohio,Texas,California
a,0.0,1.0,2.0
b,NaN,NaN,NaN
c,3.0,4.0,5.0
d,6.0,7.0,8.0


In [17]:
states = ['Texas', 'Utah', 'California']
frame.reindex(columns=states)

,Texas,Utah,California
a,1,NaN,2
c,4,NaN,5
d,7,NaN,8


In [18]:
frame.loc[['a', 'b', 'c', 'd'], states]

/Users/ramzisaouma/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  """Entry point for launching an IPython kernel.


,Texas,Utah,California
a,1.0,NaN,2.0
b,NaN,NaN,NaN
c,4.0,NaN,5.0
d,7.0,NaN,8.0


### Dropping Entries from an Axis

The __drop__ method allows you to drop a data entry as per below:

In [19]:
obj = pd.Series(np.arange(5.), index=['a', 'b', 'c', 'd', 'e'])
obj


a    0.0
b    1.0
c    2.0
d    3.0
e    4.0
dtype: float64

In [20]:
new_obj = obj.drop('c')
new_obj


a    0.0
b    1.0
d    3.0
e    4.0
dtype: float64

__drop__ method works with both columns and rows as illustrated below:

In [21]:
data = pd.DataFrame(np.arange(16).reshape((4, 4)),
                    index=['Ohio', 'Colorado', 'Utah', 'New York'],
                    columns=['one', 'two', 'three', 'four'])
data

,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7
Utah,8,9,10,11
New York,12,13,14,15


In [22]:
data.drop(['Colorado', 'Ohio'])

,one,two,three,four
Utah,8,9,10,11
New York,12,13,14,15


In [23]:
data.drop('two', axis=1)


,one,three,four
Ohio,0,2,3
Colorado,4,6,7
Utah,8,10,11
New York,12,14,15


In [24]:
data

,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7
Utah,8,9,10,11
New York,12,13,14,15


In case you want to drop the data without assigning it to a new DataFrame the optional kew argument __inplace__=True can be invoked as below (Be carefull with __inplace__, as it destroys any data that is dropped:

In [25]:
obj.drop('c', inplace=True)
obj

a    0.0
b    1.0
d    3.0
e    4.0
dtype: float64

![Methods](Pandas_methods.jpeg)

### Indexing, Selection, and Filtering

Series indexing ( obj[...] )
works similarly to NumPy array indexing, except you can use
the Series’s index values instead of only integers. Here are some
examples of this: 


In [26]:
obj = pd.Series(np.arange(4.), index=['a', 'b', 'c', 'd'])
obj


a    0.0
b    1.0
c    2.0
d    3.0
dtype: float64

In [27]:
obj['b']


1.0

In [28]:
obj[1]


1.0

In [29]:
obj[2:4]


c    2.0
d    3.0
dtype: float64

In [30]:

obj[['b', 'a', 'd']]


b    1.0
a    0.0
d    3.0
dtype: float64

In [31]:

obj[[1, 3]]


b    1.0
d    3.0
dtype: float64

In [32]:
obj[obj < 2]

a    0.0
b    1.0
dtype: float64

 #### Slicing with labels behaves differently than normal Python slicing in that the endpoint is inclusive: 



In [34]:
obj['b':'c']

b    1.0
c    2.0
dtype: float64

Values could be changed using the below familiar way:

In [36]:
obj['b':'c'] = 5
obj

a    0.0
b    5.0
c    5.0
d    3.0
dtype: float64

Indexing into a Dataframe allows the extraction of one or more columns. This is done by retreiving a single value or a sequence:

In [38]:
data = pd.DataFrame(np.arange(16).reshape((4, 4)),
                    index=['Ohio', 'Colorado', 'Utah', 'New York'],
                    columns=['one', 'two', 'three', 'four'])
data


,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7
Utah,8,9,10,11
New York,12,13,14,15


In [39]:
data['two']


Ohio         1
Colorado     5
Utah         9
New York    13
Name: two, dtype: int64

In [40]:
data[['three', 'one']]

,three,one
Ohio,2,0
Colorado,6,4
Utah,10,8
New York,14,12


Indexing like this has a few special cases. First, slicing or
selecting data with a boolean array: 


In [42]:
data[:2]


,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7


In [43]:
data[data['three'] > 5]

,one,two,three,four
Colorado,4,5,6,7
Utah,8,9,10,11
New York,12,13,14,15


The row selection syntax data[:2] is provided
as a convenience. Passing a single element or a list to the
[] operator selects columns. Another use case is in indexing with a boolean DataFrame, such as
one produced by a scalar comparison: 


In [46]:
data < 5


,one,two,three,four
Ohio,True,True,True,True
Colorado,True,False,False,False
Utah,False,False,False,False
New York,False,False,False,False


In [47]:
data[data < 5] = 0
data

,one,two,three,four
Ohio,0,0,0,0
Colorado,0,5,6,7
Utah,8,9,10,11
New York,12,13,14,15


#### Selection with loc and iloc

For DataFrame label-indexing on the rows, I introduce the
special indexing operators loc and
iloc . They enable you to select a subset of the
rows and columns from a DataFrame with NumPy-like notation using
either axis labels ( loc ) or integers
( iloc ). As a preliminary example, let’s select a single row and multiple
columns by label: 


In [49]:
data.loc['Colorado', ['two', 'three']]

two      5
three    6
Name: Colorado, dtype: int64

In [50]:
data.iloc[2, [3, 0, 1]]


four    11
one      8
two      9
Name: Utah, dtype: int64

In [51]:
data.iloc[2]


one       8
two       9
three    10
four     11
Name: Utah, dtype: int64

In [52]:
data.iloc[[1, 2], [3, 0, 1]]

,four,one,two
Colorado,7,0,5
Utah,11,8,9


Similarly we can work with slicing as below: 


In [53]:
data.loc[:'Utah', 'two']


Ohio        0
Colorado    5
Utah        9
Name: two, dtype: int64

In [54]:
data.iloc[:, :3][data.three > 5]

,one,two,three
Colorado,0,5,6
Utah,8,9,10
New York,12,13,14


As you can see DataFrame provides few different away to select and extract data for the table. Table below provides a short summary of many of them. Moreover we will see many additional options for working with hierarchical indexes. 
 

![indexing](indexing.jpeg)

### Integer Indexes

Working with pandas objects indexed by integers is something that often trips
up new users due to some differences with indexing semantics on built-in
Python data structures like lists
and tuples. For example, you might not expect the following code to
generate an error: 



```Python
ser = pd.Series(np.arange(3.))
ser
ser[-1]
```

In [62]:
ser = pd.Series(np.arange(3.))

In [64]:
ser

0    0.0
1    1.0
2    2.0
dtype: float64

In [61]:
ser[-1]

KeyError: -1

In this case, pandas could “fall back” on integer indexing, but
it’s difficult to do this in general without introducing subtle bugs.
Here we have an index containing 0, 1, 2, but inferring what the user
wants (label-based indexing or position-based) is difficult: 
On the other hand, with a non-integer index, there is no potential
for ambiguity: 


In [66]:
ser2 = pd.Series(np.arange(3.), index=['a', 'b', 'c'])
ser2[-1]

2.0

In [70]:
ser[:1]


0    0.0
dtype: float64

In [71]:
ser.loc[:1]


0    0.0
1    1.0
dtype: float64

In [72]:
ser.iloc[:1]

0    0.0
dtype: float64

### Arithmetic and Data Alignment

 When you are adding together
two DataFrames, if any index pairs are not the same, the respective index in
the result will be the union of the index pairs.

In [74]:
s1 = pd.Series([7.3, -2.5, 3.4, 1.5], index=['a', 'c', 'd', 'e'])
s2 = pd.Series([-2.1, 3.6, -1.5, 4, 3.1],
               index=['a', 'c', 'e', 'f', 'g'])


In [75]:
s1


a    7.3
c   -2.5
d    3.4
e    1.5
dtype: float64

In [76]:

s2

a   -2.1
c    3.6
e   -1.5
f    4.0
g    3.1
dtype: float64

In [77]:
s1 + s2

a    5.2
c    1.1
d    NaN
e    0.0
f    NaN
g    NaN
dtype: float64

The internal data alignment introduces missing values in the label
locations that don’t overlap. Missing values will then propagate in
further arithmetic computations. In the case of DataFrame, alignment is performed on both the rows
and the columns: 
 

In [79]:
df1 = pd.DataFrame(np.arange(9.).reshape((3, 3)), columns=list('bcd'),
                   index=['Ohio', 'Texas', 'Colorado'])
df2 = pd.DataFrame(np.arange(12.).reshape((4, 3)), columns=list('bde'),
                   index=['Utah', 'Ohio', 'Texas', 'Oregon'])


In [80]:
df1

,b,c,d
Ohio,0.0,1.0,2.0
Texas,3.0,4.0,5.0
Colorado,6.0,7.0,8.0


In [81]:
df2

,b,d,e
Utah,0.0,1.0,2.0
Ohio,3.0,4.0,5.0
Texas,6.0,7.0,8.0
Oregon,9.0,10.0,11.0


In [82]:
df1 + df2

,b,c,d,e
Colorado,NaN,NaN,NaN,NaN
Ohio,3.0,NaN,6.0,NaN
Oregon,NaN,NaN,NaN,NaN
Texas,9.0,NaN,12.0,NaN
Utah,NaN,NaN,NaN,NaN


Obviously the columns 'c' and 'e'  are not found in both DataFrame objects, hence they appear as all
missing in the result. This also applies to the rows whose labels are not
common to both DataFrames. If you add two DataFrames with no column or row labels in
common, you expect the result to have only nulls: 



In [87]:
df1 = pd.DataFrame({'A': [1, 2]})
df2 = pd.DataFrame({'B': [3, 4]})


In [88]:
df1


,A
0,1
1,2


In [89]:
df2


,B
0,3
1,4


In [90]:
df1 - df2

,A,B
0,NaN,NaN
1,NaN,NaN


#### Arithmetic methods with fill values

In case you want replace missing values with specific input:


In [93]:
df1 = pd.DataFrame(np.arange(12.).reshape((3, 4)),
                   columns=list('abcd'))
df2 = pd.DataFrame(np.arange(20.).reshape((4, 5)),
                   columns=list('abcde'))


In [94]:
df2.loc[1, 'b'] = np.nan


In [95]:
df1


,a,b,c,d
0,0.0,1.0,2.0,3.0
1,4.0,5.0,6.0,7.0
2,8.0,9.0,10.0,11.0


In [96]:
df2

,a,b,c,d,e
0,0.0,1.0,2.0,3.0,4.0
1,5.0,NaN,7.0,8.0,9.0
2,10.0,11.0,12.0,13.0,14.0
3,15.0,16.0,17.0,18.0,19.0


In [97]:
df1 + df2

,a,b,c,d,e
0,0.0,2.0,4.0,6.0,NaN
1,9.0,NaN,13.0,15.0,NaN
2,18.0,20.0,22.0,24.0,NaN
3,NaN,NaN,NaN,NaN,NaN


In [98]:
df1.add(df2, fill_value=0)

,a,b,c,d,e
0,0.0,2.0,4.0,6.0,4.0
1,9.0,5.0,13.0,15.0,9.0
2,18.0,20.0,22.0,24.0,14.0
3,15.0,16.0,17.0,18.0,19.0


See Table below for a listing of Series
and DataFrame methods for arithmetic. Each of them has a counterpart,
starting with the letter r , that has arguments
flipped. So these two statements are equivalent: 
 

In [99]:
1 / df1


,a,b,c,d
0,inf,1.000000,0.500000,0.333333
1,0.250000,0.200000,0.166667,0.142857
2,0.125000,0.111111,0.100000,0.090909


In [100]:
df1.rdiv(1)

,a,b,c,d
0,inf,1.000000,0.500000,0.333333
1,0.250000,0.200000,0.166667,0.142857
2,0.125000,0.111111,0.100000,0.090909


When reindexing a Series or DataFrame, you can also
specify a different fill value: 


In [101]:
df1.reindex(columns=df2.columns, fill_value=0)

,a,b,c,d,e
0,0.0,1.0,2.0,3.0,0
1,4.0,5.0,6.0,7.0,0
2,8.0,9.0,10.0,11.0,0


![indexing](pandas_method.jpeg)

#### Operations between DataFrame and Series

As with NumPy arrays of different dimensions, arithmetic between DataFrame and
Series is also defined. First, as a motivating example, consider the
difference between a two-dimensional array and one of its rows: 


In [106]:
arr = np.arange(12.).reshape((3, 4))
arr


array([[ 0.,  1.,  2.,  3.],
       [ 4.,  5.,  6.,  7.],
       [ 8.,  9., 10., 11.]])

In [107]:
arr[0]


array([0., 1., 2., 3.])

In [108]:
arr - arr[0]

array([[0., 0., 0., 0.],
       [4., 4., 4., 4.],
       [8., 8., 8., 8.]])

When we subtract arr[0] from arr , the subtraction is performed once for each row. This is referred to as broadcasting and is explained in more detail as it relates to general NumPy arrays in Appendix A . Operations between a DataFrame and a Series are similar: 



In [112]:
frame = pd.DataFrame(np.arange(12.).reshape((4, 3)),
                     columns=list('bde'),
                     index=['Utah', 'Ohio', 'Texas', 'Oregon'])
series = frame.iloc[0]


In [123]:
frame


,b,d,e
Utah,0.0,1.0,2.0
Ohio,3.0,4.0,5.0
Texas,6.0,7.0,8.0
Oregon,9.0,10.0,11.0


In [114]:
series

b    0.0
d    1.0
e    2.0
Name: Utah, dtype: float64

By default, arithmetic between DataFrame and Series matches the index of the Series on the DataFrame’s columns, broadcasting down the
rows: 

In [116]:
frame - series

,b,d,e
Utah,0.0,0.0,0.0
Ohio,3.0,3.0,3.0
Texas,6.0,6.0,6.0
Oregon,9.0,9.0,9.0


If an index value is not found in either the DataFrame’s columns
or the Series’s index, the objects will be reindexed to form the
union: 
 

In [117]:
series2 = pd.Series(range(3), index=['b', 'e', 'f'])
frame + series2

,b,d,e,f
Utah,0.0,NaN,3.0,NaN
Ohio,3.0,NaN,6.0,NaN
Texas,6.0,NaN,9.0,NaN
Oregon,9.0,NaN,12.0,NaN


If you want to instead broadcast over the columns, matching on
the rows, you have to use one of the arithmetic methods. For
example: 
 

In [119]:
series3 = frame['d']


In [120]:
frame


,b,d,e
Utah,0.0,1.0,2.0
Ohio,3.0,4.0,5.0
Texas,6.0,7.0,8.0
Oregon,9.0,10.0,11.0


In [121]:
series3


Utah       1.0
Ohio       4.0
Texas      7.0
Oregon    10.0
Name: d, dtype: float64

In [125]:
frame.sub(series3,axis='index')

,b,d,e
Utah,-1.0,0.0,1.0
Ohio,-1.0,0.0,1.0
Texas,-1.0,0.0,1.0
Oregon,-1.0,0.0,1.0


The axis number that you pass is the axis to match
on . In this case we mean to match on the DataFrame’s row
index ( axis='index' or axis=0 )
and broadcast across. 
 

### Function Application and Mapping

In [ ]:
frame = pd.DataFrame(np.random.randn(4, 3), columns=list('bde'),
                     index=['Utah', 'Ohio', 'Texas', 'Oregon'])
frame
np.abs(frame)

In [ ]:
f = lambda x: x.max() - x.min()
frame.apply(f)

In [ ]:
frame.apply(f, axis='columns')

In [ ]:
def f(x):
    return pd.Series([x.min(), x.max()], index=['min', 'max'])
frame.apply(f)

In [ ]:
format = lambda x: '%.2f' % x
frame.applymap(format)

In [ ]:
frame['e'].map(format)

### Sorting and Ranking

In [ ]:
obj = pd.Series(range(4), index=['d', 'a', 'b', 'c'])
obj.sort_index()

In [ ]:
frame = pd.DataFrame(np.arange(8).reshape((2, 4)),
                     index=['three', 'one'],
                     columns=['d', 'a', 'b', 'c'])
frame.sort_index()
frame.sort_index(axis=1)

In [ ]:
frame.sort_index(axis=1, ascending=False)

In [ ]:
obj = pd.Series([4, 7, -3, 2])
obj.sort_values()

In [ ]:
obj = pd.Series([4, np.nan, 7, np.nan, -3, 2])
obj.sort_values()

In [ ]:
frame = pd.DataFrame({'b': [4, 7, -3, 2], 'a': [0, 1, 0, 1]})
frame
frame.sort_values(by='b')

In [ ]:
frame.sort_values(by=['a', 'b'])

In [ ]:
obj = pd.Series([7, -5, 7, 4, 2, 0, 4])
obj.rank()

In [ ]:
obj.rank(method='first')

In [ ]:
# Assign tie values the maximum rank in the group
obj.rank(ascending=False, method='max')

In [ ]:
frame = pd.DataFrame({'b': [4.3, 7, -3, 2], 'a': [0, 1, 0, 1],
                      'c': [-2, 5, 8, -2.5]})
frame
frame.rank(axis='columns')

### Axis Indexes with Duplicate Labels

In [ ]:
obj = pd.Series(range(5), index=['a', 'a', 'b', 'b', 'c'])
obj

In [ ]:
obj.index.is_unique

In [ ]:
obj['a']
obj['c']

In [ ]:
df = pd.DataFrame(np.random.randn(4, 3), index=['a', 'a', 'b', 'b'])
df
df.loc['b']

## Summarizing and Computing Descriptive Statistics

In [ ]:
df = pd.DataFrame([[1.4, np.nan], [7.1, -4.5],
                   [np.nan, np.nan], [0.75, -1.3]],
                  index=['a', 'b', 'c', 'd'],
                  columns=['one', 'two'])
df

In [ ]:
df.sum()

In [ ]:
df.sum(axis='columns')

In [ ]:
df.mean(axis='columns', skipna=False)

In [ ]:
df.idxmax()

In [ ]:
df.cumsum()

In [ ]:
df.describe()

In [ ]:
obj = pd.Series(['a', 'a', 'b', 'c'] * 4)
obj.describe()

### Correlation and Covariance

conda install pandas-datareader

In [ ]:
price = pd.read_pickle('examples/yahoo_price.pkl')
volume = pd.read_pickle('examples/yahoo_volume.pkl')

import pandas_datareader.data as web
all_data = {ticker: web.get_data_yahoo(ticker)
            for ticker in ['AAPL', 'IBM', 'MSFT', 'GOOG']}

price = pd.DataFrame({ticker: data['Adj Close']
                     for ticker, data in all_data.items()})
volume = pd.DataFrame({ticker: data['Volume']
                      for ticker, data in all_data.items()})

In [ ]:
returns = price.pct_change()
returns.tail()

In [ ]:
returns['MSFT'].corr(returns['IBM'])
returns['MSFT'].cov(returns['IBM'])

In [ ]:
returns.MSFT.corr(returns.IBM)

In [ ]:
returns.corr()
returns.cov()

In [ ]:
returns.corrwith(returns.IBM)

In [ ]:
returns.corrwith(volume)

### Unique Values, Value Counts, and Membership

In [ ]:
obj = pd.Series(['c', 'a', 'd', 'a', 'a', 'b', 'b', 'c', 'c'])

In [ ]:
uniques = obj.unique()
uniques

In [ ]:
obj.value_counts()

In [ ]:
pd.value_counts(obj.values, sort=False)

In [ ]:
obj
mask = obj.isin(['b', 'c'])
mask
obj[mask]

In [ ]:
to_match = pd.Series(['c', 'a', 'b', 'b', 'c', 'a'])
unique_vals = pd.Series(['c', 'b', 'a'])
pd.Index(unique_vals).get_indexer(to_match)

In [ ]:
data = pd.DataFrame({'Qu1': [1, 3, 4, 3, 4],
                     'Qu2': [2, 3, 1, 2, 3],
                     'Qu3': [1, 5, 2, 4, 4]})
data

In [ ]:
result = data.apply(pd.value_counts).fillna(0)
result

## Conclusion

In [ ]:
pd.options.display.max_rows = PREVIOUS_MAX_ROWS